In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'

In [4]:
def createdframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [5]:
train=pd.DataFrame()
train['image'],train['label']=createdframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                              image     label
0      images/train\angry\15593.jpg     angry
1      images/train\angry\15594.jpg     angry
2      images/train\angry\15596.jpg     angry
3      images/train\angry\15610.jpg     angry
4      images/train\angry\15612.jpg     angry
...                             ...       ...
5813  images/train\surprise\894.jpg  surprise
5814  images/train\surprise\909.jpg  surprise
5815  images/train\surprise\960.jpg  surprise
5816  images/train\surprise\964.jpg  surprise
5817  images/train\surprise\972.jpg  surprise

[5818 rows x 2 columns]


In [7]:
test=pd.DataFrame()
test['image'],test['label']=createdframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [9]:
def extract_features(images):
    feature=[]
    for image in tqdm(images):
        img=load_img(image,grayscale=True)
        img=np.array(img)
        feature.append(img)
    feature=np.array(feature)
    feature=feature.reshape(len(feature),48,48,1)
    return feature

In [10]:
train_features=extract_features(train['image'])

  0%|          | 0/5818 [00:00<?, ?it/s]C:\Users\prajw\anaconda3\envs\tf\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(
100%|██████████| 5818/5818 [00:03<00:00, 1615.57it/s]


In [11]:
test_features=extract_features(test['image'])

100%|██████████| 7066/7066 [00:04<00:00, 1651.03it/s]


In [12]:
x_train=train_features/255.0
x_test=test_features/255.0

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [16]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [17]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512 ,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
          
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7,activation='softmax'))

In [18]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [19]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
46/46 [==============================] - 61s 1s/step - loss: 1.8497 - accuracy: 0.2398 - val_loss: 1.9468 - val_accuracy: 0.1359
Epoch 2/100
46/46 [==============================] - 58s 1s/step - loss: 1.8335 - accuracy: 0.2530 - val_loss: 1.9456 - val_accuracy: 0.1441
Epoch 3/100
46/46 [==============================] - 59s 1s/step - loss: 1.8263 - accuracy: 0.2527 - val_loss: 1.9443 - val_accuracy: 0.1441
Epoch 4/100
29/46 [=================>............] - ETA: 18s - loss: 1.8299 - accuracy: 0.2551

KeyboardInterrupt: 

In [ ]:
model_json=model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.writer(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json
json_file=open("facialemotionmodel.json","r")
model_json=json_file.read()
json_file.close()

In [ ]:
model=model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")
label=['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img=load_img(image.grayscale=True)
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0
image='image path'
print("original image is ")
img=ef(image)

In [ ]:
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
image='image path'
print("original image is ")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img,reshape(48,48),cmap='gray')